In [ ]:
!pip install -q python-terrier
import pyterrier as pt
if not pt.started():
    pt.init()

# from pyterrier.measures import *
# dataset = pt.get_dataset('msmarco_passage')

You should consider upgrading via the '/root/venv/bin/python -m pip install --upgrade pip' command.
PyTerrier 0.8.0 has loaded Terrier 5.6 (built by craigmacdonald on 2021-09-17 13:27)



In [ ]:
def msmarco_generate():
    dataset = pt.get_dataset("msmarco_passage")
    with pt.io.autoopen(dataset.get_corpus()[0], 'rt') as corpusfile:
        for l in corpusfile:
            docno, passage = l.split("\t")
            yield {'docno' : docno, 'text' : passage}

iter_indexer = pt.IterDictIndexer("./passage_index")
indexref = iter_indexer.index(msmarco_generate(), meta={})

BM25_withFeatures = pt.FeaturesBatchRetrieve(indexref, wmodel="BM25", features=["WMODEL:Tf", "WMODEL:PL2", "WMODEL:TF_IDF"])

import xgboost as xgb
params = {'objective': 'rank:ndcg', 
          'learning_rate': 0.1, 
          'gamma': 1.0, 'min_child_weight': 0.1,
          'max_depth': 6,
          'verbose': 2,
          'random_state': 42 
         }

xgb_pipe= xgb.sklearn.XGBRanker(**params)



new_xgb_pipe = BM25_withFeatures >> pt.ltr.apply_learned_model(xgb_pipe,ltr)


new_xgb_pipe.fit(dataset.get_topics('train'), dataset.get_qrels('train'), dataset.get_topics('dev'), dataset.get_qrels('dev'))

results = pt.Experiment([BM25_withFeatures, 
                        new_xgb_pipe ],
                       dataset.get_topics("test-2019"), 
                       dataset.get_qrels("test-2019"),
                       eval_metrics=["ndcg_cut_10","map"],
                       names=["BM25", "xgBoost" ]
                  )
print(results)

/shared-libs/python3.7/py-core/lib/python3.7/site-packages/ipykernel_launcher.py:9: DeprecationWarning: specifying meta and meta_lengths in IterDictIndexer.index() is deprecated, use constructor instead
  if __name__ == '__main__':
collection.tar.gz:  62%|██████▏   | 611M/987M [00:51<00:37, 10.4MiB/s]

collection.tar.gz:  78%|███████▊  | 770M/987M [01:12<00:21, 10.8MiB/s]IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=50.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)

collection.tar.gz: 100%|██████████| 987M/987M [01:45<00:00, 9.84MiB/s]


JavaException: JVM exception occurred: java.util.concurrent.ExecutionException: java.lang.NumberFormatException java.lang.RuntimeException

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=eeb7fcb4-84cb-4f4a-9830-9d1fcbc47e1d' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>